In [2]:
import pandas as pd                                                                  
import numpy as np                                                                   
from sklearn.metrics import accuracy_score                                               
from feat import Feat                                                                
                                                                                     
from glob import glob                                                                
problem = []                                                                         
solution = []                                                                        
models = []   
seeds = [5990,9625,3620,13368,2882,3570,1011,7125]
for i,dataset in enumerate([d for d in glob('data/d_lr*.csv') if 'long' not in d]):
    print('problem:',dataset)
    problem.append(dataset.split('example_patients_')[-1].split('.csv')[0])          
    df = pd.read_csv(dataset)                                                        
    ID = df['id']                                                                    
    df.drop('id',axis=1,inplace=True)                                                    
    df_X = df.drop(['classification'],axis=1)                                        
    feature_names = df_X.columns                                                     
    X = df_X.values                                                                  
    y = df['classification'].values                                                      
    zfile = dataset.split('.csv')[0] + '_long.csv'                                   
    Z = pd.read_csv(zfile)                                                           
                                                                                     
#     random_state = np.random.randint(2**14) 
    random_state = seeds[i]
    print('seed:',random_state)                                                      
    clf = Feat(max_depth=5,                                                              
               max_dim = 10,                                                          
               obj='fitness,size',                                                      
               # gens = 0,                                                             
               # pop_size = 1,                                                           
               gens = 100,                                                           
               pop_size = 100,                                                          
               max_stall = 50,                                                       
               stagewise_xo = True,                                                      
               scorer='log',                                                         
               verbosity=2,                                                              
               shuffle=True,                                                             
               ml='LR',                                                                  
               fb=0.1,                                                               
               use_batch=False,                                                       
               batch_size=100,                                                       
               classification=True,                                                      
               feature_names = ','.join(feature_names),                              
               functions="+,-,*,/,split,split_c,"            
                         "mean,median,max,min,variance,skew,kurtosis,slope,count",       
                          # "slope",                                                 
               backprop=False,                                                           
               iters=1,                                                                  
               random_state=random_state,                                                
               n_threads=10,                                                         
               split=0.5)                                                           
                                                                                     
    clf.verbosity = 2                                                                    
    clf.fit(X,y,zfile,ID.values)                                                     
    models.append(clf)                                                               
    solution.append(clf.get_representation())                                           
    print('solution archive:\n','\n'.join(models[-1].get_archive().split('\n')[:4]))
    print('accuracy score:',accuracy_score(y,clf.predict(X,zfile,np.arange(len(X)))))
                                                                                     
for prob,soln,model in zip(problem,solution,models):                                              
    print('////////////////////////////////////////')
    print('problem:',prob)                                                           
    print('solution rep:',soln)             
    print('solution model:\n',model.get_model())             
    print('solution archive:\n','\n'.join(model.get_archive().split('\n')[:4]))
    print('////////////////////////////////////////')

problem: data/d_lr_example_patients_bmi_slope-lab_max.csv
seed: 5990
update_stats
exiting update_stats
solution archive:
 complexity	fitness	fitness_v	eqn
2	0.512287	0.939343	[max(z_lab)]
10	0.248990	1.915890	[mean(z_lab)][slope(z_bmi)]
12	0.209925	1.962407	[mean(z_lab)][slope(z_bmi)][min(z_lab)]
accuracy score: 0.755
problem: data/d_lr_example_patients_bmi_slope-lab_median-age_max.csv
seed: 9625
update_stats
exiting update_stats
solution archive:
 complexity	fitness	fitness_v	eqn
2	0.532542	0.902363	[min(z_age)]
4	0.357249	1.369502	[mean(z_age)][median(z_lab)]
12	0.096973	2.676769	[median(z_lab)][slope(z_bmi)][max(z_age)]
accuracy score: 0.531
problem: data/d_lr_example_patients_bmi_slope-lab_slope.csv
seed: 3620
update_stats
exiting update_stats
solution archive:
 complexity	fitness	fitness_v	eqn
8	0.511182	1.000051	[slope(z_lab)]
16	0.347759	2.488124	[slope(z_lab)][slope(z_bmi)]
26	0.311903	1.501914	[slope(z_lab)][(slope(z_bmi)<0.000527)]
accuracy score: 0.761
problem: data/d_lr_exa

In [5]:
for prob,soln,model in zip(problem,solution,models):                                              
    print('//////////////')
    print('problem:',prob)                                                           
    print('solution rep:',soln)             
    print('solution model:\n',model.get_model())             
    print('solution archive:\n','\n'.join(model.get_archive().split('\n')[:4]))
    print('//////////////')

//////////////
problem: bmi_slope-lab_max
solution rep: [median(z_lab)]
solution model:
 Feature		Weight
median(z_lab)		3.005189

solution archive:
 complexity	fitness	fitness_v	eqn
2	0.297664	1.596772	[mean(z_lab)]
6	0.181910	1.562213	[(max(z_lab)<1.142489)]
8	0.137040	2.168169	[max(z_lab)][(max(z_lab)<1.142489)]
//////////////
//////////////
problem: bmi_slope-lab_median-age_max
solution rep: [max(z_age)]
solution model:
 Feature		Weight
max(z_age)		-1.764653

solution archive:
 complexity	fitness	fitness_v	eqn
2	0.485758	0.959275	[max(z_age)]
10	0.258365	2.032825	[max(z_age)][slope(z_bmi)]
12	0.190911	2.449583	[min(z_age)][slope(z_bmi)][max(z_lab)]
//////////////
//////////////
problem: bmi_slope-lab_slope
solution rep: [(slope(z_bmi)<0.000068)]
solution model:
 Feature		Weight
(slope(z_bmi)<0.000068)		3.530112

solution archive:
 complexity	fitness	fitness_v	eqn
8	0.357814	2.368123	[slope(z_bmi)]
18	0.128436	1.672166	[(slope(z_bmi)<0.000000)]
20	0.122968	1.711690	[min(z_lab)][(slop